Cropped Patch Generation and Augmentation without Patchify

In [2]:
import cv2
import os
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt

def generate_crops(image):
    cropped_images = []

    height_upto = int(image.shape[0] / 2)
    width_upto = int(image.shape[1] / 2)

    # 1st crop - upper left
    cropped_upper_left = image[0:height_upto, 0:width_upto]
    cropped_images.append(cropped_upper_left)

    # 2nd crop - upper right
    cropped_upper_right = image[0:height_upto, width_upto:image.shape[1]]
    cropped_images.append(cropped_upper_right)
    # 3rd crop - lower left

    cropped_lower_left = image[height_upto:image.shape[0], 0:width_upto]
    cropped_images.append(cropped_lower_left)

    # 4th crop - lower right
    cropped_lower_right = image[height_upto:image.shape[0], width_upto:image.shape[1]]
    cropped_images.append(cropped_lower_right)

    return cropped_images


def generate_augmentations(image):
    augmented_images = []

    # Rotate Right
    rotate_right = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    augmented_images.append(rotate_right)

    # Rotate Left
    rotate_left = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    augmented_images.append(rotate_left)

    # Flip Vertical
    flip_vertical = cv2.flip(image, 0)
    augmented_images.append(flip_vertical)

    # Flip Horizontal
    flip_horizontal = cv2.flip(image, 1)
    augmented_images.append(flip_horizontal)

    return augmented_images


source = 'original'
destination = 'original_cropped'
image_count = 0

for file in tqdm(os.listdir(source)):
    if(file.endswith(".jpg")):
        # Read the main image
        image = cv2.imread(f"{source}/{file}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Generate crops from main image
        cropped_images = generate_crops(image)

        # Send each crop for augmentation
        for i in range(len(cropped_images)):
            # Generate augmentations for eacg crop
            augmented_images = generate_augmentations(cropped_images[i])
            augmented_images.append(cropped_images[i])
            # Save each image
            for j in range(len(augmented_images)):
                filename = f"{image_count}.jpg"
                image_to_save = cv2.cvtColor(augmented_images[j], cv2.COLOR_BGR2RGB)
                cv2.imwrite(f"{destination}/{filename}", image_to_save)
                image_count += 1


100%|██████████| 10/10 [00:09<00:00,  1.00it/s]


Cropped Patch Generation and Augmentation with Patchify

In [2]:
from tqdm import tqdm
import os
import cv2
from patchify import patchify
import numpy as np
#Open image using Image module

def generate_augmentations(image):
    augmented_images = []
    augmented_images.append(image)

    # Rotate Right
    rotate_right = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    augmented_images.append(rotate_right)

    # Rotate Left
    rotate_left = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    augmented_images.append(rotate_left)

    # # Flip Vertical
    flip_vertical = cv2.flip(image, 0)
    augmented_images.append(flip_vertical)

    # # Flip Horizontal
    flip_horizontal = cv2.flip(image, 1)
    augmented_images.append(flip_horizontal)

    return augmented_images

source = 'edited'
destination = 'edited_cropped'
image_count = 0

for file in tqdm(os.listdir(source)):
    if(file.endswith(".jpg")):
        image = cv2.imread(f"{source}/{file}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Cropping 374 x 374 x 3 sized Images
        images_patches = patchify(image, (374,374,3), step=374) 
        sent_to_val = 0

        for i in range(images_patches.shape[0]):
            for j in range(images_patches.shape[1]):
                single_patch_image = images_patches[i,j,:,:]
                image_to_save = cv2.cvtColor(single_patch_image[0,:,:,:], cv2.COLOR_BGR2RGB)
                augmented_images = generate_augmentations(image_to_save)
                for idx in range(len(augmented_images)):
                    cv2.imwrite(destination + '/' + str(image_count) + ".jpg", augmented_images[idx])
                    image_count += 1


100%|██████████| 11/11 [00:08<00:00,  1.32it/s]
